In [15]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [16]:
from skforecast.recursive import ForecasterRecursive
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [17]:
y = pd.Series(np.arange(10000, dtype=float), index=pd.date_range("2020-01-01", periods=10000, freq="h"))
forecaster = ForecasterRecursive(
    regressor=LinearRegression(),
    transformer_y=StandardScaler(),
    lags=100
)

In [ ]:
x_train, y_train = forecaster._create_lags(y=y.to_numpy())
x_train_old, y_train_old = forecaster._create_lags_deprecated(y=y.to_numpy())
np.testing.assert_array_equal(x_train, x_train_old)
np.testing.assert_array_equal(y_train, y_train_old)

In [19]:
# %%timeit
# forecaster.fit(y=series)

In [20]:
# %%timeit
# forecaster.fit(y=series)

In [21]:
%%timeit
forecaster._create_lags_deprecated(
    y
)

5.7 ms ± 458 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit
forecaster._create_lags(
    y
)

784 μs ± 39.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
from skforecast.recursive import ForecasterRecursiveMultiSeries

forecaster = ForecasterRecursiveMultiSeries(
    regressor=LinearRegression(),
    lags=3,
)

series = {
    f"series_{i}": pd.Series(
        np.arange(1000, dtype=float), index=pd.date_range("2020-01-01", periods=1000)
    )
    for i in range(1000)
}

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : skforecast.exceptions.DataTransformationWarning                           │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/recursive/_forecaster_recursive │
│ _multiseries.py:487                                                                  │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [24]:
for y in series.values():
    x_train, y_train = forecaster._create_lags(y=y.to_numpy())
    x_train_old, y_train_old = forecaster._create_lags_deprecated(y=y.to_numpy())
    np.testing.assert_array_equal(x_train, x_train_old)
    np.testing.assert_array_equal(y_train, y_train_old)

In [25]:
%%timeit

for y in series.values():
    forecaster._create_lags_deprecated(
        y
    )

159 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit

for y in series.values():
    forecaster._create_lags(
        y
    )

81.3 ms ± 3.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
%%timeit

forecaster.fit(series=series)

6.59 s ± 95.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%%timeit

forecaster.fit(series=series)

5.92 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
from skforecast.direct import ForecasterDirect

forecaster = ForecasterDirect(
    regressor=LinearRegression(),
    steps=24,
    lags=50
)
y = pd.Series(np.arange(1000, dtype=float), index=pd.date_range("2020-01-01", periods=1000))


In [28]:
x_train, y_train = forecaster._create_lags(y=y.to_numpy())
x_train_old, y_train_old = forecaster._create_lags_deprecated(y=y.to_numpy())
np.testing.assert_array_equal(x_train, x_train_old)
np.testing.assert_array_equal(y_train, y_train_old)

In [29]:
%%timeit
forecaster._create_lags_deprecated(
    y
)

2.7 ms ± 49.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit
forecaster._create_lags(
    y
)

67.1 μs ± 3.13 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [37]:
forecaster = ForecasterRecursive(LinearRegression(), lags=3)
results = forecaster._create_lags(
    y=np.arange(10, dtype=float), X_as_pandas=False, 
    train_index=pd.date_range('2020-01-03', periods=7, freq='D')
)
results

(array([[2., 1., 0.],
        [3., 2., 1.],
        [4., 3., 2.],
        [5., 4., 3.],
        [6., 5., 4.],
        [7., 6., 5.],
        [8., 7., 6.]]),
 array([3., 4., 5., 6., 7., 8., 9.]))